In [1]:
from pathlib import Path
import pyart
import numpy as np
from scipy.ndimage import uniform_filter1d
import copy
import pandas as pd

import xarray as xr
import cv2



In [1]:

from prepro.nexrad import prune_nexrad
from cappi.azran import make_cappi
from cappi.make_vv import simple_vv
from mcit.vil import get_vil_from_azran

#viewable plots
import matplotlib.pyplot as plt
from klaus_krause_cmap import get_zdr_cmap
from config import _EXAMPLEDATA_DIR




## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



ImportError: cannot import name 'indexed_vv' from 'cappi.make_vv' (F:\BOKU\CIWRO\hotspots\cappi\make_vv.py)

In [3]:
filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KUDX20150620_040849_V06.gz')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KOAX20140603_213649_V06.gz')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KEWX20210504_020040_V06')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KSHV20230613_230228_V06')
#filename = Path_EXAMPLEDATA_DIR, 'nexrad_level2', 'KTLX20160526_215113_V06')
#filename = Path(_EXAMPLEDATA_DIR, 'nexrad_level2', 'KTLX20200422_023145_V06')
#read in the Level 2 WSR-88D data as a pyart object
radar_vol = pyart.io.read_nexrad_archive(filename)
radar_vol.info()


In [4]:
#remove extra sweeps of data. Keep only data
#from the survelience cuts and one cut per volume 
prune_actions = ['surv', 'volume']
prune_vol = prune_nexrad(prune_actions, radar_vol)

#from hotspots.io.radarinfo import get_project_root, read_nexrad_radarinfo
#print(get_project_root())
#radar_info = read_nexrad_radarinfo()
#print(radar_info)


In [5]:
import matplotlib.pyplot as plt
display = pyart.graph.RadarDisplay(prune_vol)

display.plot_ppi('reflectivity', sweep=0)
plt.show()

In [6]:
#w_phase = pyart.correct.unwrap.dealias_unwrap_phase(prune_vol, unwrap_unit='volume')
#rune_vol.add_field("unwrapped_differential_phase", uw_phase)
prune_vol.info()


In [7]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

In [8]:
def linear_interp(data, first_index, last_index, verbose=False):
    #send data and two indexs, returns data with linear interploation
    start_data = data[first_index]
    end_data = data[last_index]
    if end_data == np.nan:
        end_data = start_data
        
    dist = last_index - first_index
    if verbose:
        print('fi: ', first_index, ' li: ', last_index, 'start_data: ', start_data, ' end: ', end_data)
    for i in range(first_index, last_index, 1):
        data[i] = start_data + (end_data - start_data)*(i-first_index)/(last_index-first_index)
        if verbose:
            print(i, ' fi: ', first_index, ' li: ', last_index, 'start_data: ', start_data, ' end: ', end_data, ' ', data[i])

In [11]:
#
#Eventually our goal here is to create the entire nORPG PreProcessor stack, but at
#the moment we need the work done on the reflectivity and the zdr fields, which is
#easier and faster than the computation of KDP. So skipping the KDP computation
#we replicate the smoothing and attenuation corrections done on the ORPG
#
#we use the nORPG name to identify that we are targeting the ORPG, but we cannot
#stay totally upto date with it.
#
from pyart.util.sigmath import rolling_window
from prepro.process_phase import unwrap_phase, compute_min_system_phase, correct_system_phase
np.set_printoptions(suppress=True)
#basic plan:
#  unwrap_phi
pyart_vol = prune_vol
#
sweeps = pyart_vol.sweep_number['data']

#extract a sweep of phi and rhv

#print('working...will notify at end')
si= 100
ei = 200
ia = 16000
#our output target
uw_phi = copy.deepcopy(pyart_vol.fields['differential_phase'])

cc_field = copy.deepcopy(pyart_vol.fields['cross_correlation_ratio'])

sm_phase = copy.deepcopy(pyart_vol.fields['differential_phase'])
#fixme
#check for matching azimuth in rhv_field and adjust it to match the phi_field
#right now assume they match in azimuth
missing_value = cc_field["_FillValue"]    


#for s in range(len(sweeps)):
for s in range(1):
    #which data do we use?
    start_index, end_index = prune_vol.get_start_end(s)
    print(s, 'sweep_num: ', sweeps[s], 'start: ', start_index, ' end: ', end_index)
    #our output target
    radar_sweep = pyart_vol.extract_sweeps(sweeps=[s])
    #compute_min_system_phase
    uw_phi_data = np.array(radar_sweep.fields['differential_phase']['data'])
    cc_data = np.array(radar_sweep.fields['cross_correlation_ratio']['data'])
    
    corr_phi, min_system_phase = correct_system_phase(uw_phi_data, cc_data)
    #min_system_phase = compute_min_system_phase(uw_phi['data'], cc_field['data'], missing_value=-9999.0)
    #print('min_sys_phase: ', min_sys_phase)
    
    for a in range(radar_sweep.nrays):
        #print ('a: ', a)
        #we unwrap phase like the ORPG here
        phi_radial_raw = corr_phi[a]
        num_gates = len(phi_radial_raw)-1
        if a == ia:
            print('phi_raw', phi_radial_raw[si:ei])
        
        #phi = np.where(phi_radial_raw<0, np.nan, phi_radial_raw)
        #threshold based on cc
        cc = np.array(cc_data[a])
        
        #We threshold by cc, but there are many ways to threshold the phi data
        #that sort the data into good and bad regions. 
        #The ORPG uses the MetSignal Alg to do this.
        #
        
        phase_window = 9
        phase_std_thresh=10
        cc_thresh=0.7
        #
        window_size = phase_window
        half_ws = int(np.floor(window_size/2))
        
        uw_phase = copy.deepcopy(phi_radial_raw)
        ray = np.ma.std(rolling_window(uw_phase[:],window_size), 1)
        
        phase_std = np.zeros_like(phi_radial_raw)
        phase_std[half_ws:-half_ws] = ray
        phase_std[0:half_ws] = np.ones(half_ws) * ray[0]
        phase_std[-half_ws:] = np.ones(half_ws) * ray[-1]

        uw_phase = copy.deepcopy(phi_radial_raw)
        ray = np.ma.median(rolling_window(uw_phase[:],window_size), 1)
        
        phase_median = np.zeros_like(phi_radial_raw)
        phase_median[half_ws:-half_ws] = ray
        phase_median[0:half_ws] = np.ones(half_ws) * ray[0]
        phase_median[-half_ws:] = np.ones(half_ws) * ray[-1]

        phase = copy.deepcopy(phase_median)
        #threshold off the bad data, keep the good data
        phase[phase_std > phase_std_thresh] = np.nan
        phase[cc < cc_thresh] = np.nan

        valid_idx = np.where(~np.isnan(phase))[0]
        valid_intervals = consecutive(valid_idx)
        
        if a== ia:
            print('phase_median: ', phase_median[0:25])
            print('phase_std: ', phase_std[0:25])
            print('cc: ', cc[0:25])
            print('phase: ', phase[0:25])
            print('valid_idx: ', valid_idx[0:25])
            print('valid: ', valid_intervals)
            
        #
        #This is where things get messy. Each tuple in the valid_intervals
        #can be trimmed to be conservative or not. How much to trim is also 
        #a bit of a guess. The edges of the good data intervals can be 
        #inaccurate causing negative values of phase, which are objectionable.
        #Another way to smooth out 
        #the phase is to require that the new interval start near where the old
        #interval ended. Which is conceptually pleasing, but hard because there
        #can be a strong positive bias at the end of the intervals. 
        min_interval_length = 9
        final_int = []
        for interval in valid_intervals:
            if len(interval) >= phase_window:
                #possibly trim the interval here to remove
                # a few gates on the front and back to be conservative
                # usually half the min_interval_length (4) or 0.25 of min_interval_length (2)
                #we'll keep this one
                final_int.append(interval)        
                
        #push the last value on to get interplation to the end of the radial   
        #final_int.append([num_gates-1, num_gates-1])
        
        if a == ia:                                 
            print('final_int: ',final_int)
        #phi_final = np.zeros_like(phi)
        #phi[:] = min_system_phase
      
        last_int_idx = 0
        phase[0] = 0
        for f in final_int:
            #linearly interpret the phase along the radial from the last_phase_value
            #to the first phase value of the new interval.
            if a == ia:
                linear_interp(phase, last_int_idx, f[0], False)
            else:
                linear_interp(phase, last_int_idx, f[0], False)
            last_int_idx = f[-1]
           
        if len(final_int) > 0:
            last_int = final_int[-1]
            last_valid = last_int[-1]
            last_value = phase[last_valid]
            phase[last_valid:-1] = last_value
        else:
            phase[:] = 0.0
        
        if a == ia:
            print('last_int:', last_int)
            print('last_valid: ', last_valid)
            print('last_value', last_value)
            print('cc: ', cc[0:100])
            print('initial: ', phi_radial_raw[0:100])
            print('phase: ',phase[0:100])
            
        sm_phase['data'][a] = phase
        

In [12]:
#add the unwrapped Phi to the pyart object:
prune_vol.add_field('processed_phase', sm_phase, replace_existing=True)
prune_vol.info()

In [13]:
gatefilter = pyart.correct.GateFilter(prune_vol)
gatefilter.exclude_below('cross_correlation_ratio', 0.85)

cor_diff_phase_pyart = pyart.correct.phase_proc.get_phidp_unf_gf(
    prune_vol, gatefilter)

# optional for smoothing
smoothed_diffphase = pyart.correct.phase_proc.smooth_masked(
    cor_diff_phase_pyart)
prune_vol.add_field_like('differential_phase',
                          'unwrapped_differential_phase_pyart',
                          smoothed_diffphase, replace_existing=True)


In [14]:
display = pyart.graph.RadarDisplay(prune_vol)

# sweep comparison
fig = plt.figure(figsize=(14, 4))
xlim = [0, 300]
ylim = [-50, 250]
vmax = 400
ax1 = fig.add_subplot(131)
display.plot_ppi('differential_phase', sweep=0, vmin=0.1, vmax=vmax, mask_outside=True,
                 colorbar_flag=False, title_flag=False)
display.set_limits(xlim, ylim)
display.plot_colorbar(extend='both', shrink=0.9)
ax1.set_title('Original NEXRAD Level II')

ax2 = fig.add_subplot(132)
display.plot_ppi('processed_phase', sweep=0,
                 cmap='pyart_Wild25', vmin=0.1, vmax=vmax, mask_outside=True,
                 colorbar_flag=False, title_flag=False)
display.set_limits(xlim, ylim)

display.plot_colorbar(extend='both', shrink=0.9)
ax2.set_title("JK Advanced Computing Facility")

ax3 = fig.add_subplot(133)
display.plot_ppi('unwrapped_differential_phase_pyart', sweep=0,
                 cmap='pyart_Wild25', vmin=0.1, vmax=vmax, mask_outside=True,
                 colorbar_flag=False, title_flag=False)
display.set_limits(xlim, ylim)
display.plot_colorbar(extend='both', shrink=0.9)
ax3.set_title('Py-ART unwrapping')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect('equal')

time0 = pyart.util.datetime_from_radar(prune_vol)
radarname = prune_vol.metadata['instrument_name']
fig.suptitle(radarname + str(time0), fontweight='bold', fontsize=20)

plt.tight_layout()

plt.show()

